# <font color="#76b900">Retrieval-Augmented Generation with Vector Stores</font>

## 1. Environment Setup

In [ ]:
%pip install -q langchain langchain-nvidia-ai-endpoints gradio rich
%pip install -q arxiv pymupdf faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.3 MB/s eta 0:00:00


In [ ]:
%pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00


In [ ]:
from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

In [ ]:
import os
os.environ["NVIDIA_API_KEY"] = "NVIDIA_API_KEY"

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")

## RAG Workflow
`Vector Store Workflow for Conversational Exchanges`:
* Generate semantic embedding for each new conversation.
* Add the message body to a vector store for retrieval.
* Query the vector store for relevant messages to fill in the LLM context.

`Modified Workflow for an Arbitrary Document`
* Divide the document into chunks and process them into useful messages.
* Generate semantic embedding for each new document chunk.
* Add the chunk bodies to a vector store for retrieval.
* Query the vector store for relevant chunks to fill in the LLM context

`Extended Workflow for a Directory of Arbitrary Documents`:
* Divide each document into chunks and process them into useful messages.
* Generate semantic embedding for each new document chunk.
* Add the chunk bodies to a scalable vector database for fast retrieval.
- Query the **vector database** for relevant chunks to fill in the LLM context.


# RAG for Conversation History
## 1. Constructing Our Vector Store Retriever

In [ ]:
conversation = [  ## This conversation was generated partially by an AI system, and modified to exhibit desirable properties
    "[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the rocky mountains?",
    "[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch across North America",
    "[Beras] Wow, that sounds amazing! Ive never been to the Rocky Mountains before, but Ive heard many great things about them.",
    "[Agent] I hope you get to visit them someday, Beras! It would be a great adventure for you!"
    "[Beras] Thank you for the suggestion! Ill definitely keep it in mind for the future.",
    "[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research online or watching documentaries about them."
    "[Beras] I live in the arctic, so I'm not used to the warm climate there. I was just curious, ya know!",
    "[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains and their significance!"
]

In [ ]:
%%time
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain.vectorstores import FAISS

## Streamlined from_texts FAISS vectorstore construction from text list
convstore = FAISS.from_texts(conversation, embedding=embedder)
retriever = convstore.as_retriever()

CPU times: user 80 ms, sys: 8.08 ms, total: 88 ms
Wall time: 507 ms


In [ ]:
pprint(retriever.invoke("What is your name?"))

[
    Document(
        id='35c17900-46da-4617-8d70-3c301cb97a4e',
        metadata={},
        page_content="[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the 
rocky mountains?"
    ),
    Document(
        id='31028f7a-74cb-433a-bcf5-2a57c51c96cf',
        metadata={},
        page_content='[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains
and their significance!'
    ),
    Document(
        id='86d49355-c97b-4b77-a0be-af6b26a6e1bb',
        metadata={},
        page_content='[Agent] I hope you get to visit them someday, Beras! It would be a great adventure for 
you![Beras] Thank you for the suggestion! Ill definitely keep it in mind for the future.'
    ),
    Document(
        id='2f548416-99c5-45bd-9038-b166451dd89f',
        metadata={},
        page_content="[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research 
online or watching documentaries about them.[Beras] I live in the arctic, so I'm not used to the warm climate 
there. I was just curious, ya know!"
    )
]

In [ ]:
pprint(retriever.invoke("Where are the Rocky Mountains?"))

[
    Document(
        id='e5de6eaf-bdd0-4d95-a4a3-1aa9c1d079d2',
        metadata={},
        page_content='[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch 
across North America'
    ),
    Document(
        id='2f548416-99c5-45bd-9038-b166451dd89f',
        metadata={},
        page_content="[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research 
online or watching documentaries about them.[Beras] I live in the arctic, so I'm not used to the warm climate 
there. I was just curious, ya know!"
    ),
    Document(
        id='35c17900-46da-4617-8d70-3c301cb97a4e',
        metadata={},
        page_content="[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the 
rocky mountains?"
    ),
    Document(
        id='31028f7a-74cb-433a-bcf5-2a57c51c96cf',
        metadata={},
        page_content='[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains
and their significance!'
    )
]

## 2. Incorporating Conversation Retrieval Into Our Chain

In [ ]:
from langchain.document_transformers import LongContextReorder
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain.schema.runnable.passthrough import RunnableAssign
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from functools import partial
from operator import itemgetter

## Utility Runnables/Methods
def RPrint(preface=""):
    """Simple passthrough "prints, then returns" chain"""
    def print_and_return(x, preface):
        if preface: print(preface, end="")
        pprint(x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))


In [ ]:
def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name:
            out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

## Optional; Reorders longer documents to center of output text
long_reorder = RunnableLambda(LongContextReorder().transform_documents)

In [ ]:
context_prompt = ChatPromptTemplate.from_template(
    "Answer the question using only the context"
    "\n\nRetrieved Context: {context}"
    "\n\nUser Question: {question}"
    "\nAnswer the user conversationally. User is not aware of context."
)

chain = (
    {
        'context': convstore.as_retriever() | long_reorder | docs2str,
        'question': (lambda x:x)
    }
    | context_prompt
    # | RPrint()
    | instruct_llm
    | StrOutputParser()
)

pprint(chain.invoke("Where does Beras live?"))

It seems that Beras lives in the Arctic. The cooler climate there is quite different from the warm climate found in
the Rocky Mountains, which Beras mentioned was curious about.

In [ ]:
pprint(chain.invoke("Where are the Rocky Mountains?"))

Hello there! The Rocky Mountains are a stunning range of mountains that span across North America. Unfortunately, 
without more detailed information, I can't provide a more specific location, but I can tell you that they're known 
for their breathtaking beauty and significance. You might want to look into them further or watch documentaries to 
learn more about their exact location and other fascinating details!

In [ ]:
pprint(chain.invoke("Where are the Rocky Mountains? Are they close to California?"))

Hello there! The Rocky Mountains are a stunning range of mountains that span across North America. To answer your 
question about their location, they do not run directly through California but are quite close. They extend from 
northern British Columbia in Canada all the way down to New Mexico in the United States. So, while they are not 
directly in California, they are relatively close to the state.

In [ ]:
pprint(chain.invoke("How far away is Beras from the Rocky Mountains?"))

While I can't provide specifics on Beras's exact location, I can tell you that since Beras mentioned living in the 
Arctic, they are quite a distance away from the Rocky Mountains. The Arctic is located at the northernmost part of 
the Earth, while the Rocky Mountains are primarily in western North America. So, it's safe to say that Beras is 
indeed far away from the Rocky Mountains.

## 3. Automatic Conversation Storage

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

In [ ]:
## Reset knowledge base and define what it means to add more messages.
convstore = FAISS.from_texts(conversation, embedding=embedder)

def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([f"User said {d.get('input')}", f"Agent said {d.get('output')}"])
    return d.get('output')

In [ ]:
chat_prompt = ChatPromptTemplate.from_template(
    "Answer the question using only the context"
    "\n\nRetrieved Context: {context}"
    "\n\nUser Question: {input}"
    "\nAnswer the user conversationally. Make sure the conversation flows naturally.\n"
    "[Agent]"
)


In [ ]:
conv_chain = (
    {
        'context': convstore.as_retriever() | long_reorder | docs2str,
        'input': (lambda x:x)
    }
    | RunnableAssign({'output' : chat_prompt | instruct_llm | StrOutputParser()})
    | partial(save_memory_and_get_output, vstore=convstore)
)

pprint(conv_chain.invoke("I'm glad you agree! I can't wait to get some ice cream there! It's such a good food!"))
print()
pprint(conv_chain.invoke("Can you guess what my favorite food is?"))
print()
pprint(conv_chain.invoke("Actually, my favorite is honey! Not sure where you got that idea?"))
print()
pprint(conv_chain.invoke("I see! Fair enough! Do you know my favorite food now?"))

While I'm sure the Rocky Mountains would provide a magnificent backdrop for enjoying some ice cream, the context 
only mentions the scenic beauty and significance of the Rockies, not any food options. However, regardless of where
you are, ice cream is always a delightful treat, isn't it? We could perhaps discuss the best spots for viewing the 
mountains while savoring a scoop or two. What do you think?

Well, based on our conversation, I'd hazard a guess that you might have a fondness for ice cream! It certainly 
seems like a treat you're excited about. But remember, this is just a speculation based on our chat. Everyone's 
taste buds are different, and that's what makes food such an interesting topic!

I apologize for the misunderstanding! It's fascinating to know that your favorite food is actually honey, not ice 
cream as I initially speculated. Everyone's unique taste preferences are what make our culinary conversations so 
engaging! If you'd like, we can explore the Rockies while enjoying some honey-flavored treats. Doesn't that sound 
like a sweet adventure?

I sure do! After our little chat, I can confidently say that your favorite food is honey. And you know what? I 
found it absolutely fascinating to learn that. Everyone's taste preferences are unique, and discovering new 
favorites is always an exciting adventure. In fact, we can even imagine exploring the Rockies with some 
honey-flavored treats! How does that sound?